In [1]:
import re
import pandas as pd
import spacy
import skweak

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('./CSVs/V1/REL_AVISO_LICITACAO.csv', dtype=str)

In [3]:
data

,Unnamed: 0.1,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,0,0,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,modalidade_licitacao,1,anotador_22346437,432851,17,PREGAO ELETRONICO,12_22.11.2018-R21
1,1,1,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,numero_licitacao,2,anotador_22346437,432872,8,116/2018,12_22.11.2018-R21
2,2,2,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,valor_estimado_contratacao,6,anotador_22346437,433432,10,"767.907,37",12_22.11.2018-R21
3,3,3,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,data_abertura_licitacao,7,anotador_22346437,433599,10,20/11/2018,12_22.11.2018-R21
4,4,4,12_22.11.2018,REL_AVISO_LICITACAO,R21,anotador_22346437,sistema_compras,8,anotador_22346437,433716,21,www.compras.df.gov.br,12_22.11.2018-R21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7408,7408,40994,638,REL_AVISO_LICITACAO,R22,anotador_57860312,valor_estimado_contratacao,3880,anotador_57860312,191759,13,"51.518.031,96",638-R22
7409,7409,40995,638,REL_AVISO_LICITACAO,R22,anotador_57860312,data_abertura_licitacao,3881,anotador_57860312,191804,19,25 de abril de 2022,638-R22
7410,7410,40996,638,REL_AVISO_LICITACAO,R22,anotador_57860312,nome_responsavel,3882,anotador_57860312,192207,27,LADÉRCIO BRITO SANTOS FILHO,638-R22
7411,7411,40997,638,REL_AVISO_LICITACAO,R22,anotador_57860312,objeto_licitacao,3923,anotador_57860312,190998,705,"contratação pelo Distrito Federal, por meio da...",638-R22


In [4]:
#cria um novo dataset apenas com o tipo de entidade EXTATO_CONTRATO (TEXTO)
data = data.loc[data['tipo_ent'] == 'AVISO_LICITACAO'].reset_index(drop=True)



In [5]:
#criando novo dataframe somente com os textos
df = data['texto'].reset_index(drop=True)
df

0      AVISOS DE LICITACOES PREGAO ELETRONICO No 116/...
1      PREGAO ELETRONICO No 129/2018 OBJETO: Registro...
2      AVISOS DE LICITACOES PREGAO ELETRONICO No 141/...
3      PREGAO ELETRONICO No 142/2018-UASG 925041 Obje...
4      AVISO DE ABERTURA PREGAO ELETRONICO POR SRP No...
                             ...                        
633    AVISO DE RETIFICAÇÃO (*) CONCORRÊNCIA Nº 003/2...
634    AVISO CONCORRÊNCIA Nº 02/2022    Processo:    ...
635    AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 07/202...
636    AVISO DE LICITAÇÃO - CONCORRÊNCIA Nº 03/2022 A...
637    AVISO DE RETOMADA DA LICITAÇÃO Concorrência nº...
Name: texto, Length: 638, dtype: object

In [6]:
#df = ["EXTRATO DO CONTRATO No 11/2019 PROCESSO: 00390-00003006/2019-92. ESPECIE: Contrato de Aquisicao de Bens pelo Distrito Federal no 11/2019. PARTES: Secretaria de Estado de Desenvolvimento Urbano e Habitacao do Distrito Federal - SEDUH/DF e INDUSTRIA DE AGUA MINERAL IBIA LTDA. DO OBJETO: Aquisicao de agua potavel de mesa sem gas e garrafao retornavel, destinados ao atendimento desta pasta, conforme demais especiGcacoes e condicoes estabelecidas no Termo de Referencia Anexo I do Edital de Pregao Eletronico no 0058/2018 - SCG/SEPLAG, da Proposta da empresa INDUSTRIA DE AGUA MINERAL IBIA LTDA de 11/06/2018 e da Lei no 8.666, de 21 de junho de 1993, que passam a integrar o presente Termo. DO VALOR: R$ 11.661,00 (onze mil, seiscentos e sessenta e um reais) conforme Nota de Empenho no 2019NE00180, emitida em 18/07/2019. Dotacao Orcamentaria P.T 15.122.6001.8517.0131 - Natureza da Despesa: 3390.30 - Fonte de Recursos: 100. DO PRAZO DE VIGENCIA: O Contrato tera vigencia de 06 (seis) meses a contar do dia 07/08/2019. DATA DE ASSINATURA: 22/07/2019. SIGNATARIOS: Pelo Distrito Federal: Mateus Leandro de Oliveira, na qualidade de Secretario de Estado. Pela Contratada: Eduardo Barros de Queiroz Rodrigues, na qualidade de Socio Administrador.", "EXTRATO DO CONTRATO Nº 06/2021 Processo: 00139-00001275/2021-00. Partes: DF - ADMINISTRAÇÃO REGIONAL DO CRUZEIRO X COMERCIAL DE ALIMENTOS PARK SUL. Objeto: O presente contrato tem por objeto a contratação de empresa especializada na área de alimentação para confecção e fornecimento de 50 KG DE BOLO CONFEITADO TIPO ANIVERSÁRIO COM MASSA PÃO DE LÓ COM RECHEIO DE BABA DE MOÇA, BEIJINHO OU DOCE DE LEITE E COBERTURA DE CHANTILLY DECORADO COM ACABAMENTO, no evento de comemoração ao aniversário de 62 anos do Cruzeiro no dia 30/11/2021. Valor do Contrato: R$ 2.000,00 (Dois Mil Reais). Dotação orçamentária: Unidade Orçamentária: 190113. Programa de Trabalho: 13.392.6219.3678.0136 . Natureza da Despesa: 339039. Fonte de Recursos: 120. Nota de Empenho: 2021NE00259. Valor do Empenho Inicial: R$ 2.000,00 (Dois Mil Reais), emitida em 26/11/2021 sob o evento 400091, na Modalidade 1   Ordinário. Fundamento Legal: Art. 24 da Lei nº 8.666/1993. Data da Assinatura: 29/11/2021. Signatários: Pelo DISTRITO FEDERAL: LUIZ EDUARDO GOMES DE PAULA PESSOA, na qualidade de Administrador Regional do Cruzeiro; pela COMERCIAL DE ALIMENTOS PARK SUL: GIVANILDES DE AGUIAR MORAIS, na qualidade de Gerente Geral."] 

In [7]:
# CODIGO RESPONSAVEL PELA LIMPEZA DO TEXTO DA BASE OURO
import re

df1 =[]

for i in df:
    i = i.replace("\n", " ")
    i = re.sub('xx[a-z]{1,10}', '', i)
    #i = i.replace(r"xx[a-z]{1,10}","")
    aux = ' '.join(i.split())
    #print(aux)
    df1.append(aux)

In [8]:
# Carregando nlp do spacy em pt-br sem o ner e o lemmatizer
nlp = spacy.load('pt_core_news_sm', disable=["ner", "lemmatizer"])
docs = list(nlp.pipe(df1))

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [9]:
#docs

In [15]:
def processo_(doc):
        '''
        label function para extracao de processo usando regex

        parametros:
            doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''
        expression = r"[P|p][R|r][O|o][C|c][E|e][S|s][S|s][O|o][\s\S].*?[s|:]?.+?(?=[0-9]).*?(\d*[^;|,|a-zA-Z]*)"    
        match = re.search(expression, str(doc))
        if match:
                    flag = 0
                    for token in doc:
                        if match.span(1)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                            if(doc[token.i].text == ':'):
                                start = doc[token.i+1]
                            else:
                                start = token
                            flag = 1
                        if token.idx >= match.span(1)[1] and flag == 1 and token.i > start.i:
                            if(doc[token.i-1].text in ['.', '-', '—', ':']):
                                end = doc[token.i-1]
                            else:
                                end = token
                            yield start.i, end.i, "processo_gdf"
                            break    

def modalidade_(doc):
        '''
        label function para extracao de modalidade de licitação usando regex

        parametros:
            doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''
        modalidade = r"(?:PREG[AÃ]O|[Pp]reg[aã]o)\s(?:ELETR[OÔ]NICO|[Ee]letr[oô]nico)|(?:CONCORR[EÊ]NCIA|[Cc]oncorr[eê]ncia)|(?:TOMADA|[Tt]omada)\s[Dd][Ee]\s(?:PRE[CÇ]OS?|[Pp]re[cç]os?)|(?:CARTA\s|[Cc]arta\s)?(?:CONVITE|[Cc]onvite)|(?:CONCURSO|[Cc]oncurso)|(?:LEIL[AÃ]O|[Ll]eil[aã]o)"

        match = re.finditer(modalidade, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(0)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(0)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "modalidade_licitacao"
                        break               
def numero_licitacao_(doc):
        '''
        label function para extracao de modalidade de licitação usando regex

        parametros:
            doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''
       
        
        expression = r"((?:PREG[AÃ]O|[Pp]reg[aã]o)\s(?:ELETR[OÔ]NICO|[Ee]letr[oô]nico)|(?:CONCORR[EÊ]NCIA|[Cc]oncorr[eê]ncia)|(?:TOMADA|[Tt]omada)\s[Dd][Ee]\s(?:PRE[CÇ]OS?|[Pp]re[cç]os?)|(?:CARTA\s|[Cc]arta\s)?(?:CONVITE|[Cc]onvite)|(?:CONCURSO|[Cc]oncurso)|(?:LEIL[AÃ]O|[Ll]eil[aã]o))[\s\S][N|n][\s|o|º|.].+?(?=[0-9].*?)(\d*[^;|,|a-zA-Z]*?)"     
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "numero_licitacao"
                        break          
        
        expression = r"((?:CONCORR[E|Ê]NCIA|[Cc]oncorr[eê]ncia)\s(?:P[ÚU]BLICA|[Pp][úu]blica))[\s\S][N|n][\s|o|º|.].+?(?=[0-9].*?)(\d*[^;|,|a-zA-Z]*?)"
             
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        #print(token, "1")
                        
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                            #print(token, "2")
                        else:
                            start = token
                            print(token, "3")
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "numero_licitacao"
                        break
        expression = r"((?:PREG[AÃ]O|[Pp]reg[aã]o)\s(?:ELETR[OÔ]NICO|[Ee]letr[oô]nico)\s(?:[Ss][Rr][Pp]))[\s\S][N|n][\s|o|º|.].+?(?=[0-9].*?)(\d*[^;|,|a-zA-Z]*?)"                
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "numero_licitacao"
                        break        
        #RDC Nº 02/2021 
        expression = r"([Rr][Dd][Cc])[\s\S][N|n][\s|o|º|.].+?(?=[0-9].*?)(\d*[^;|,|a-zA-Z]*?)"                
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "numero_licitacao"
                        break 
                    
        expression = r"((?:PREG[AÃ]O|[Pp]reg[aã]o)\s(?:ELETR[OÔ]NICO|[Ee]letr[oô]nico)\s(?:[Pp][Oo][Rr]|[Dd][Ee]|\s)\s(?:[Ss][Rr][Pp]))[\s\S][N|n][\s|o|º|.].+?(?=[0-9].*?)(\d*[^;|,|a-zA-Z]*?)"                
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "numero_licitacao"
                        break                    
        
        expression = r"((?:PREG[AÃ]O|[Pp]reg[aã]o)\s(?:ELETR[OÔ]NICO|[Ee]letr[oô]nico)\s(?:[Ii][Nn][Tt][Ee][Rr][Nn][Aa][Cc][Ii][Oo][Nn][Aa][Ll]))[\s\S][N|n][\s|o|º|.].+?(?=[0-9].*?)(\d*[^;|,|a-zA-Z]*?)"                
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "numero_licitacao"
                        break
        
        expression = r"(?:(?:DISPENSA\sDE\s|[Dd]ispensa\s[Dd]e\s)|)(?:LICITA[CÇ][AÃ]O|[Ll]icita[cç][aã]o)[\s\S][N|n][\s|o|º|.].+?(?=[0-9].*?)(\d*[^;|,|a-zA-Z]*?)"                     
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(1)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(1)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "numero_licitacao"
                        break
def valor_(doc):
        '''
            label function para extracao do valor estimado usando regex

            parametros:
                doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''
        #Valor Estimado
        expression = r"[V|v][a|A][l|L][o|O][r|R](\d*[^;|,|a-zA-Z]*?)[E|e][S|s][T|t][I|i][M|m][A|a][D|d][O|o].*?[\s\S].*?([\d\.]*,\d{2})"
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "valor_estimado_contratacao"
                        break
        
'''         #Valor Total Estimado
        expression = r"[V|v][a|A][l|L][o|O][r|R](\d*[^;|,|a-zA-Z]*?)[T|t][O|o][T|t][A|a][L|l](\d*[^;|,|a-zA-Z]*?)[E|e][S|s][T|t][I|i][M|m][A|a][D|d][O|o].*?[\s\S].*?([\d\.]*,\d{2})"
       
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(3)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(3)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "valor_estimado_contratacao"
                        break
        
        
        #Valor Global Estimado
        expression = r"[V|v][a|A][l|L][o|O][r|R](\d*[^;|,|a-zA-Z]*?)[G|g][L|l][O|o][B|b][A|a][L|l](\d*[^;|,|a-zA-Z]*?)[E|e][S|s][T|t][I|i][M|m][A|a][D|d][O|o].*?[\s\S].*?([\d\.]*,\d{2})"
       
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(3)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(3)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "valor_estimado_contratacao"
                        break
       
        #Valor Anual Estimado
        expression = r"[V|v][a|A][l|L][o|O][r|R](\d*[^;|,|a-zA-Z]*?)[A|a][N|n][U|u][L|l](\d*[^;|,|a-zA-Z]*?)[E|e][S|s][T|t][I|i][M|m][A|a][D|d][O|o].*?[\s\S].*?([\d\.]*,\d{2})"
       
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(3)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(3)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "valor_estimado_contratacao"
                        break            
        expression = r"[D|d][O|o][\s\S][V|v][a|A][l|L][o|O][r|R].*?[\s\S].*?([\d\.]*,\d{2})"
        match = re.finditer(expression, str(doc))
        if match:
            for grupo in match:
                flag = 0
                for token in doc:
                    if grupo.span(2)[0]+1 in range(token.idx, (token.idx+len(token))+1) and flag == 0:
                        if(doc[token.i].text == ':'):
                            start = doc[token.i+1]
                        else:
                            start = token
                        flag = 1
                    if token.idx >= grupo.span(2)[1] and flag == 1 and token.i > start.i:
                        if(doc[token.i-1].text in ['.', '-', '—', ':']):
                            end = doc[token.i-1]
                        else:
                            end = token
                        yield start.i, end.i, "valor_estimado_contratacao"
                        break  '''  
def valor_fun(doc):
        '''
        label function para extracao de valor com comparacoes de listas

        parametros:
            doc: uma string respresentando o texto de um dos contratos oferecidos no vetor da base de dados
        '''
        #Valor Estimado
        for token in doc:
            if token.i+3 < len(doc):
                #print("token.i+3", token)
                for y in ['ESTIMADO', 'Estimado', 'estimado','ESTIMADO:', 'Estimado:', 'estimado:']:
                    if y in token.text:
                        for x in range(1, len(doc)-token.i-2):
                            if (doc[token.i+x].text in ['R$', '$', '$$'] and doc[token.i+x+1].text[0].isdigit()) and doc[token.i+x].i+1 < doc[token.i+x].i+2:
                                #print("doc[token.i+x].i+1", doc[token.i+x].i+1)
                                yield doc[token.i+x].i+1, doc[token.i+x].i+2,"valor_estimado_contratacao",
                                break
            #Valor Total Estimado
            if token.i+4 < len(doc):
                for y in ['VALOR', 'Valor', 'valor']:
                    if y in token.text and doc[token.i+1].text in ['TOTAL', 'Total', 'total'] and doc[token.i+2].text in ['ESTIMADO', 'Estimado', 'estimado','ESTIMADO:', 'Estimado:', 'estimado:']:
                        for x in range(1, len(doc)-token.i-2):
                                if (doc[token.i+x].text in ['R$', '$', '$$'] and doc[token.i+x+1].text[0].isdigit()) and doc[token.i+x].i+1 < doc[token.i+x].i+2:
                                    #print("doc[token.i+x].i+1", doc[token.i+x].i+1)
                                    yield doc[token.i+x].i+1, doc[token.i+x].i+2,"valor_estimado_contratacao",
                                    break 
            #Valor Global Estimado
            if token.i+4 < len(doc):
                for y in ['VALOR', 'Valor', 'valor']:
                    if y in token.text and doc[token.i+1].text in ['GLOBAL', 'Global', 'global'] and doc[token.i+2].text in ['ESTIMADO', 'Estimado', 'estimado','ESTIMADO:', 'Estimado:', 'estimado:']:
                        for x in range(1, len(doc)-token.i-2):
                                if (doc[token.i+x].text in ['R$', '$', '$$'] and doc[token.i+x+1].text[0].isdigit()) and doc[token.i+x].i+1 < doc[token.i+x].i+2:
                                    #print("doc[token.i+x].i+1", doc[token.i+x].i+1)
                                    yield doc[token.i+x].i+1, doc[token.i+x].i+2,"valor_estimado_contratacao",
                                    break                        
            #Valor Anual Estimado
            if token.i+4 < len(doc):
                for y in ['VALOR', 'Valor', 'valor']:
                    if y in token.text and doc[token.i+1].text in ['ANUAL', 'Anual', 'anual'] and doc[token.i+2].text in ['ESTIMADO', 'Estimado', 'estimado','ESTIMADO:', 'Estimado:', 'estimado:']:
                        for x in range(1, len(doc)-token.i-2):
                                if (doc[token.i+x].text in ['R$', '$', '$$'] and doc[token.i+x+1].text[0].isdigit()) and doc[token.i+x].i+1 < doc[token.i+x].i+2:
                                    #print("doc[token.i+x].i+1", doc[token.i+x].i+1)
                                    yield doc[token.i+x].i+1, doc[token.i+x].i+2,"valor_estimado_contratacao",
                                    break                             


''' 
     Aplica a label function para extracao do processo do ato de suspensão de licitação
'''                    
detec_processo = skweak.heuristics.FunctionAnnotator("detec_processo", processo_)
docs = list(detec_processo.pipe(docs))

'''
     Aplica a label function para extracao de modalidade de licitação do ato de suspensão de licitação
'''                     
detec_modalidade = skweak.heuristics.FunctionAnnotator("detec_modalidade", modalidade_)
docs = list(detec_modalidade.pipe(docs))

'''
     Aplica a label function para extracao do número de licitação do ato de suspensão de licitação 
'''                     
detec_numero_licitacao = skweak.heuristics.FunctionAnnotator("detec_numero_licitacao", numero_licitacao_)
docs = list(detec_numero_licitacao.pipe(docs))


detec_valor = skweak.heuristics.FunctionAnnotator("detec_valor", valor_)
docs = list(detec_valor.pipe(docs))

valor_detector_fun = skweak.heuristics.FunctionAnnotator("valor_detector_fun", valor_fun)
docs = list(valor_detector_fun.pipe(docs))

            
'''
     Exibe o contrato com os rótulos gerados pelas Labels Functions
'''                    
#all_fun = ["detec_processo","detec_modalidade", "detec_numero_licitacao", "detec_valor","detec_valor_detector_fun"]
all_fun = ["detec_processo","detec_modalidade", "detec_numero_licitacao", "valor_detector_fun"]
for i in range(len(docs)):
    skweak.utils.display_entities(docs[i], all_fun)


03/2022 3
